In [54]:
import requests
from bs4 import BeautifulSoup

In [238]:
def get_base_page(url):
    response = requests.get(url)

    if response.status_code == 200:
        print('Success!')
        text = response.text
    else:
        print(f'Error: {response.status_code}')
    soup = BeautifulSoup(text, 'lxml')
    return soup

def get_teachers_links(soup):
    teachers_links = []
    page_list = soup.find('div', class_='catalog-listing')
    teachers = page_list.find_all('div', class_='list-card-body')
    for row in page_list.find_all('a', class_='list-card-link'):
        teachers_links.append(row.get('href'))
    return teachers_links

def get_teacher_page_content(teacher_url):
    print(teacher_url)
    response_teacher = requests.get(teacher_url)

    if response_teacher.status_code == 200:
        print('Success!')
        text_teacher = response_teacher.text
    else:
        print(f'Error: {response_teacher.status_code}')

    soup_teacher = BeautifulSoup(text_teacher, 'lxml')
    teacher_aside = soup_teacher.find('div', class_='teacher-aside')
    teacher_content = soup_teacher.find('div', class_='teacher-content')
    return teacher_aside, teacher_content

def get_teacher_stat(teacher_stat, teacher_dict):
    teacher_dict['students'] = 0
    teacher_dict['num_lessons'] = 0
    teacher_dict['reit'] = 0
    teacher_dict['experience'] = 0
    for i in range(len(teacher_stat)):
        if teacher_stat[i].span.text == 'довольныхученика' or teacher_stat[i].span.text == 'довольныхучеников' or teacher_stat[i].span.text == 'довольныйученик':
            teacher_dict['students'] = int(teacher_stat[i].find('b').text.replace(' ', ''))
        if teacher_stat[i].span.text == 'урокав Skyeng' or teacher_stat[i].span.text == 'уроковв Skyeng' or teacher_stat[i].span.text == 'урокв Skyeng':
            teacher_dict['num_lessons'] = int(teacher_stat[i].find('b').text.replace(' ', ''))
        if teacher_stat[i].span.text == 'рейтингсреди учеников':
            teacher_dict['reit'] = float(teacher_stat[i].find('b').text.split('из')[0].replace(' ', ''))
        if teacher_stat[i].span.text == 'летопыта':
            teacher_dict['experience'] = int(teacher_stat[i].find('b').text.replace(' ', ''))
    return teacher_dict


def parse_teacher_info(teacher_aside, teacher_content):
    teacher_dict = {}
    teacher_dict['name'] = teacher_content.find('h1', class_='teacher-name').text
    teacher_dict['price'] = teacher_content.find('div', class_='teacher-price').text
    benefits = []
    for row in teacher_content.find('ul', class_='teacher-ul ng-star-inserted').find_all('li'):
        benefits.append(row.text)
    teacher_dict['benefits'] = benefits    
    
    teacher_stat = teacher_content.find_all('ul')[1].find_all('li')
    teacher_dict = get_teacher_stat(teacher_stat, teacher_dict)

    levels = []
    for row in teacher_content.find_all('div', class_='level-item ng-star-inserted'):
        levels.append(row.text)
    teacher_dict['levels'] = levels
    teacher_dict['about'] = teacher_content.find('div', class_='teacher-text -desktop').text

    teachers_tags = []
    teachers_interests = []
    for row in teacher_content.find_all('div', class_='tags')[0].find_all('span'):
        teachers_tags.append(row.text)
    for row in teacher_content.find_all('div', class_='tags')[1].find_all('span'):
        teachers_interests.append(row.text)
    teacher_dict['teachers_tags'] = teachers_tags
    teacher_dict['teachers_interesrs'] = teachers_interests

    img_url = teacher_aside.find('img')['src']
    img_name = img_url.split('/')[3]
    download_img(img_url, img_name)
    teacher_dict['img_path'] = img_name
    return teacher_dict

def download_img(img_url, img_name):
    print(f'--- download image {img_url} ---')
    print(f'--- saved to {img_folder + img_name + ".jpeg"} ---')
    img_file = requests.get(img_url).content
    with open(img_folder + img_name + '.jpeg', 'wb') as f:
        f.write(img_file)

def one_teacher_process(teacher_link):
    teacher_url = 'https://skyeng.ru/' + teacher_link    
    teacher_aside, teacher_content = get_teacher_page_content(teacher_url)    
    teacher_dict = parse_teacher_info(teacher_aside, teacher_content)
    teacher_dict['url'] = teacher_url
    return teacher_dict

In [243]:
teachers_dict = {}
page_url_base = 'https://skyeng.ru/teachers/'
img_folder = '/home/asergeeva/Desktop/parser/skyeng_info/images/'
page_counter = 0
for page_num in range(1, 4):
    print(f'\n--- page {page_num} ---\n')
    if page_num == 1:
        page_url_postfix = '?from=main_new_menu'
    else:
        page_url_postfix = f'page-{page_num}/?from=main_new_menu'   
    page_url = page_url_base + page_url_postfix   
    print('page url: ', page_url)
    soup = get_base_page(page_url)
    teachers_links = get_teachers_links(soup)
    for i_teacher in range(0, len(teachers_links), 2):
        print(f'--- run {i_teacher} ---')
        teacher_link = teachers_links[i_teacher]
        key = teacher_link.split('/')[3]
        teacher_dict = one_teacher_process(teacher_link)
        print(teacher_dict)
        teachers_dict[key] = teacher_dict
    print(' ------ ')



--- page 1 ---

page url:  https://skyeng.ru/teachers/?from=main_new_menu
Success!
--- run 0 ---
https://skyeng.ru//teachers/details/20348907/?from=main_new_menu
Success!
летопыта
довольныйученик
уроковв Skyeng
рейтингсреди учеников
--- download image https://auth-avatars-skyeng.imgix.net/20348907/af265b1b-e113-4a18-a13c-314500276a66?auto=format%2Cenhance&fit=crop&h=350&w=350&q=100&sat=10&sharp=40&vib=50 ---
--- saved to /home/asergeeva/Desktop/parser/skyeng_info/images/20348907.jpeg ---
{'name': 'Annita', 'price': 'от 3247 ₽ за урок  Native ', 'benefits': ['Есть опыт проживания за границей', 'Сертификат TEFL/TESOL', 'Эксперт в курсах Изучение английского языка для общих целей, Разговорный английский язык, Для бизнеса'], 'students': 51, 'num_lessons': 759, 'reit': 4.8, 'experience': 8, 'levels': ['B1Intermediate', 'B2Upper-intermediate', 'C1Advanced'], 'about': 'My name is Annita Jacobs and English native from South Africa. I am an online teacher by profession and have been for over 6